Data Extraction

In [49]:
import requests
import pandas as pd

# FRESH COOKIE MUST BE PROVIDED
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
  'Cookie': 'dwanonymous_6ff516fc64e5194a34356fc83db5a3b8=aecibM4A3FS6kaPLZoUzhUvfMe; dwsid=QU5Uglb2H3Sw55wxTeJnVUn3BotAPMYaWSLBktW0xZgju1x8dp4LaKp9OakqqblUwVrEFJ3liqDVQengL70wLw==; sid=V-aoF6vQ2vUnknz2u9NK7l9f-h69GgObzlc'
}
base_url = 'https://www.moncler.com/on/demandware.store/Sites-MonclerUS-Site/en_US/SearchApi-Search?'
#headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"}
params = {
    'sz': 12,  # Size of the page
    'start': 0  # Starting index
}
total_products = None
all_products = []


In [50]:

while True:
    try:
        response = requests.get(base_url, params=params,headers=headers)
        response.raise_for_status()
        data = response.json()

        # Check if 'data' key exists and contains 'products'
        if 'data' in data and 'products' in data['data']:
            products = data['data']['products']
        else:
            print("No products found in the response.")
            break

        if total_products is None:
            total_products = data['data'].get('count', 0)

        for product in products:
            processed_product = {
                'masterId' : product.get('masterId',''),
                'style' : product.get('id',''),
                'category': product.get('analyticsAttributes', {}).get('category', ''),
                'season': product.get('analyticsAttributes', {}).get('dimension22', ''),
                'gender': product.get('analyticsAttributes', {}).get('dimension38', ''),
                'subCategory':product.get('analyticsAttributes', {}).get('dimension39', ''),
                'productName': product.get('productName', ''),
                'shortDescription': product.get('shortDescription', ''),
                'productCharacteristics': product.get('productCharacteristics', '').replace('<ul>', '').replace('</ul>', '').replace('<li>', '').replace('</li>', '; '),
                'price': product.get('price', {}).get('sales', {}).get('formatted', ''),
                'colors': '; '.join([v.get('displayValue', '') for v in product.get('variationAttributes', []) if v.get('attributeId') == 'color']),
                'images': '; '.join(product.get('imgs', {}).get('urls', [])),
                'pUrl':product.get('productUrl',""),
                'modelName':product.get('originalModelName',""),


            }
            all_products.append(processed_product)

        # update to fetch next page
        params['start'] += 12
        if params['start'] >= total_products:
            break

    except requests.HTTPError as e:
        print(f"HTTP error: {e}")
        break
    except requests.RequestException as e:
        print(f"Request error: {e}")
        break
    except ValueError as e:
        print(f"JSON decode error: {e}")
        break


df = pd.DataFrame(all_products)


df.to_csv('moncler_products.csv', index=False)

Data Pre Proccessing

In [53]:

df = pd.read_csv('moncler_products.csv')
df.head()




,masterId,style,category,season,gender,subCategory,productName,shortDescription,productCharacteristics,price,colors,images,pUrl,modelName
0,I20931C00037595FE,I20931C00037595FE999,outerwear,cont,women,long coat/car coat,Hirmafur Long Down Jacket,Luxury meets warmth in the Hirmafur down coat ...,Crafted from longue saison; Longue saison lini...,"$3,300.00",Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/long-down-jackets/hirma...,Hirmafur
1,J10931A1010053048,J10931A1010053048999,outerwear,cont,women,jackets,Lans Short Down Jacket,"Designed to transcend seasons and trends, the ...",Crafted from nylon léger; Nylon léger lining; ...,"$1,150.00",Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/short-down-jackets/lans...,Lans Short Down Jacket
2,J10911A0009054155,J10911A0009054155743,outerwear,cont,men,jackets,Grimpeurs Hooded Jacket,"Inspired by the world of boating, the Grimpeur...",Crafted from nylon technique; Adjustable hood ...,$895.00,Dark Blue,https://moncler-cdn.thron.com/delivery/public/...,/en-us/men/outerwear/windbreakers-and-raincoat...,Grimpeurs Hooded Jacket
3,I20931A0014268950,I20931A0014268950999,outerwear,cont,women,jackets,Maire Short Down Jacket,The Maire is an iconic Moncler womenswear piec...,Crafted from nylon laqué; Nylon laqué lining; ...,"$1,995.00",Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/short-down-jackets/mair...,Maire
4,J10911A00139549SK,J10911A00139549SK999,outerwear,ss24,men,gilet,Lechtal Down Vest,"A hard-working wardrobe addition, the Lechtal ...",Crafted from lightweight micro chic; Lightweig...,$950.00,Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/men/outerwear/vests/lechtal-down-vest-b...,Lechtal Down Vest


In [69]:
df.describe()

,masterId,style,category,season,gender,subCategory,productName,shortDescription,productCharacteristics,price,colors,images,pUrl,modelName
count,1739,1739,1739,1739,1739,1739,1739,1739,1739,1545,1739,1739,1739,1739
unique,1178,1739,12,8,5,88,855,1092,1123,314,106,1739,1739,852
top,J10918A7030084556,I20931C00037595FE999,outerwear,ss24,men,jackets,Logo T-Shirt,"A timeless design that transcends seasons, thi...",Crafted from cotton piquet; Tricolor knit coll...,$385.00,Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/long-down-jackets/hirma...,Logo T-Shirt
freq,9,1,617,1016,682,329,48,13,9,40,582,1,1,49


In [137]:
#I am going to create a datafram with needed columns
filtered_data = df[['category', 'modelName', 'season', 'gender', 'colors']]


In [141]:
import re

def clean_model_name(name):
    name = name.lower()

    name = re.sub(r'[^a-zA-Z\s]', '', name)

    name = re.sub(r'\s+', ' ', name)

    name = name.strip()

    return name


filtered_data['modelName'] = filtered_data['modelName'].apply(clean_model_name)

<ipython-input-141-5ddf8c7d714b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['modelName'] = filtered_data['modelName'].apply(clean_model_name)


In [145]:
filtered_data['modelName'].unique()

array(['hirmafur', 'lans short down jacket', 'grimpeurs hooded jacket',
       'maire', 'lechtal down vest', 'hiengu rain coat', 'avocette',
       'logo patch polo shirt', 'vion', 'swim shorts',
       'alfit short down jacket', 'treompan', 'argenno short down jacket',
       'gabardine baseball cap', 'authie', 'benamou short down jacket',
       'logo motif polo shirt', 'galene padded jacket', 'cotton beanie',
       'felicie', 'gui down vest', 'double logo patch tshirt',
       'basile slides', 'leda parka', 'acorus down jacket', 'montcla',
       'parana', 'trick backpack', 'tiarna crossbody bag',
       'fegeo hooded jacket', 'logo tshirt', 'ghany',
       'new pierrick backpack', 'gles down jacket',
       'vertical logo tshirt', 'durance belt bag', 'bormes',
       'logo patch swim shorts', 'joly hooded jacket', 'bernard',
       'maurienne short down jacket', 'mayaf', 'tib',
       'cashmere beanie with pom pom', 'beanie with pom pom',
       'padded wool cardigan', 'amiot', 'm

In [146]:
filtered_data.dropna(inplace=True)

<ipython-input-146-d4050551feac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.dropna(inplace=True)


In [147]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1739 entries, 0 to 1738
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   category   1739 non-null   object
 1   modelName  1739 non-null   object
 2   season     1739 non-null   object
 3   gender     1739 non-null   object
 4   colors     1739 non-null   object
dtypes: object(5)
memory usage: 68.1+ KB


In [148]:
# Convert categorical variables to dummy variables
data_encoded = pd.get_dummies(filtered_data, columns=['season', 'gender', 'colors','modelName'])


In [149]:
data_encoded.columns

Index(['category', 'season_cont', 'season_fw20', 'season_fw22', 'season_fw23',
       'season_fw24', 'season_ss15', 'season_ss23', 'season_ss24',
       'gender_boy',
       ...
       'modelName_wool taffeta cardigan', 'modelName_wool turtleneck sweater',
       'modelName_wrap skirt', 'modelName_wrapid',
       'modelName_yazi short down jacket', 'modelName_zene vest',
       'modelName_zipup hoodie', 'modelName_zipup polo shirt',
       'modelName_zipup sweatshirt', 'modelName_zipup wool cardigan'],
      dtype='object', length=968)

In [150]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1739 entries, 0 to 1738
Columns: 968 entries, category to modelName_zipup wool cardigan
dtypes: bool(967), object(1)
memory usage: 1.6+ MB


In [152]:
#SPLITTING DATA INTO train and test
from sklearn.model_selection import train_test_split


X = data_encoded.drop('category', axis=1)
y = data_encoded['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [176]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000))

# Fit the pipeline on the selected training data
pipeline.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=2000))])

In [177]:
from sklearn.metrics import accuracy_score

y_pred = pipeline.predict(X_test)

y_pred = pipeline.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.83


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

baseline_models = [
    {
        'name': 'Logistic Regression',
        'model': LogisticRegression(max_iter=1000)
    },
    {
        'name': 'SVM',
        'model': SVC()
    },
    {
        'name': 'Decision Tree',
        'model': DecisionTreeClassifier()
    },
    {
        'name': 'Random Forest',
        'model': RandomForestClassifier()
    },
    {
        'name': 'K-Nearest Neighbors',
        'model': KNeighborsClassifier()
    },
    {
        'name': 'Gaussian Naive Bayes',
        'model': GaussianNB()
    },
    {
        'name': 'Linear Discriminant Analysis',
        'model': LinearDiscriminantAnalysis()
    }
]

baseline_results = []

for model_dict in baseline_models:
    model_name = model_dict['name']
    model = model_dict['model']

    pipeline = make_pipeline(StandardScaler(), model)

    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    baseline_results.append({
        'name': model_name,
        'accuracy': accuracy
    })

print("Baseline Model Comparison:")
print("Model\tAccuracy")
for result in baseline_results:
    print(f"{result['name']}\t{result['accuracy']:.2f}")

best_baseline_model = max(baseline_results, key=lambda x: x['accuracy'])

print("\nBest Baseline Model:")
print(f"Model: {best_baseline_model['name']}")
print(f"Accuracy: {best_baseline_model['accuracy']:.2f}")

print("\nLogistic Regression Improvement:")

param_grid = {
    'logisticregression__C': [0.1, 1.0, 10.0],
    'logisticregression__solver': ['lbfgs', 'saga']
}

pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Best Logistic Regression Model Accuracy: {accuracy:.2f}")
print(f"Best Logistic Regression Hyperparameters: {grid_search.best_params_}")

Baseline Model Comparison:
Model	Accuracy
Logistic Regression	0.84
SVM	0.63
Decision Tree	0.66
Random Forest	0.66
K-Nearest Neighbors	0.40
Gaussian Naive Bayes	0.70
Linear Discriminant Analysis	0.58

Best Baseline Model:
Model: Logistic Regression
Accuracy: 0.84

Logistic Regression Improvement:


KeyboardInterrupt: 

LOOKS LIKE IT MAKES SENSE TO KEEP linear regression. i cannot improve the accuracy using purely machine learning model optimization strategies. However, I will try to optimize the training data set

In [184]:

df = pd.read_csv('moncler_products.csv')
df.head()




,masterId,style,category,season,gender,subCategory,productName,shortDescription,productCharacteristics,price,colors,images,pUrl,modelName
0,I20931C00037595FE,I20931C00037595FE999,outerwear,cont,women,long coat/car coat,Hirmafur Long Down Jacket,Luxury meets warmth in the Hirmafur down coat ...,Crafted from longue saison; Longue saison lini...,"$3,300.00",Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/long-down-jackets/hirma...,Hirmafur
1,J10931A1010053048,J10931A1010053048999,outerwear,cont,women,jackets,Lans Short Down Jacket,"Designed to transcend seasons and trends, the ...",Crafted from nylon léger; Nylon léger lining; ...,"$1,150.00",Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/short-down-jackets/lans...,Lans Short Down Jacket
2,J10911A0009054155,J10911A0009054155743,outerwear,cont,men,jackets,Grimpeurs Hooded Jacket,"Inspired by the world of boating, the Grimpeur...",Crafted from nylon technique; Adjustable hood ...,$895.00,Dark Blue,https://moncler-cdn.thron.com/delivery/public/...,/en-us/men/outerwear/windbreakers-and-raincoat...,Grimpeurs Hooded Jacket
3,I20931A0014268950,I20931A0014268950999,outerwear,cont,women,jackets,Maire Short Down Jacket,The Maire is an iconic Moncler womenswear piec...,Crafted from nylon laqué; Nylon laqué lining; ...,"$1,995.00",Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/short-down-jackets/mair...,Maire
4,J10911A00139549SK,J10911A00139549SK999,outerwear,ss24,men,gilet,Lechtal Down Vest,"A hard-working wardrobe addition, the Lechtal ...",Crafted from lightweight micro chic; Lightweig...,$950.00,Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/men/outerwear/vests/lechtal-down-vest-b...,Lechtal Down Vest


In [186]:
new_df = df
new_df

,masterId,style,category,season,gender,subCategory,productName,shortDescription,productCharacteristics,price,colors,images,pUrl,modelName
0,I20931C00037595FE,I20931C00037595FE999,outerwear,cont,women,long coat/car coat,Hirmafur Long Down Jacket,Luxury meets warmth in the Hirmafur down coat ...,Crafted from longue saison; Longue saison lini...,"$3,300.00",Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/long-down-jackets/hirma...,Hirmafur
1,J10931A1010053048,J10931A1010053048999,outerwear,cont,women,jackets,Lans Short Down Jacket,"Designed to transcend seasons and trends, the ...",Crafted from nylon léger; Nylon léger lining; ...,"$1,150.00",Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/short-down-jackets/lans...,Lans Short Down Jacket
2,J10911A0009054155,J10911A0009054155743,outerwear,cont,men,jackets,Grimpeurs Hooded Jacket,"Inspired by the world of boating, the Grimpeur...",Crafted from nylon technique; Adjustable hood ...,$895.00,Dark Blue,https://moncler-cdn.thron.com/delivery/public/...,/en-us/men/outerwear/windbreakers-and-raincoat...,Grimpeurs Hooded Jacket
3,I20931A0014268950,I20931A0014268950999,outerwear,cont,women,jackets,Maire Short Down Jacket,The Maire is an iconic Moncler womenswear piec...,Crafted from nylon laqué; Nylon laqué lining; ...,"$1,995.00",Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/outerwear/short-down-jackets/mair...,Maire
4,J10911A00139549SK,J10911A00139549SK999,outerwear,ss24,men,gilet,Lechtal Down Vest,"A hard-working wardrobe addition, the Lechtal ...",Crafted from lightweight micro chic; Lightweig...,$950.00,Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/men/outerwear/vests/lechtal-down-vest-b...,Lechtal Down Vest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,I20963B00006M2538,I20963B00006M2538M99,soft accessories,fw23,unisex,helmets,Logo Ski Helmet,"Keeping you safe while you conquer the slopes,...",Crafted from nylon; ABS shell; Buckle fastenin...,$570.00,Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/men/accessories/ski-helmets/logo-ski-he...,Logo Ski Helmet
1735,I20963B00006M2538,I20963B00006M2538041,soft accessories,cont,unisex,helmets,Logo Ski Helmet,"Keeping you safe while you conquer the slopes,...",Crafted from nylon; ABS shell; Buckle fastenin...,$570.00,White,https://moncler-cdn.thron.com/delivery/public/...,/en-us/men/accessories/ski-helmets/logo-ski-he...,Logo Ski Helmet
1736,I20973A0000253066,I20973A0000253066999,soft accessories,cont,men,gloves and mittens,Padded Gloves,"Made for the slopes, these padded gloves keep ...",Crafted from nylon; Nylon lining; Padded; Zipp...,$815.00,Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/men/accessories/scarves-and-gloves/padd...,Padded Gloves
1737,I20960G00003M4327,I20960G00003M4327999,other goods,cont,unisex,accessories,Thermal Bottle,"Whether you are skiing in the winter, or trekk...",Crafted from stainless steel; Tritan drinking ...,$300.00,Black,https://moncler-cdn.thron.com/delivery/public/...,/en-us/women/grenoble/ski-accessories/thermal-...,Thermal Bottle


In [187]:
#I am going to create a datafram with needed columns
new_df = new_df[['category', 'modelName', 'season', 'gender', 'colors']]


In [189]:
import re

def clean_model_name(name):
    name = name.lower()

    name = re.sub(r'[^a-zA-Z\s]', '', name)

    name = re.sub(r'\s+', ' ', name)

    name = name.strip()

    return name


new_df['modelName'] = new_df['modelName'].apply(clean_model_name)

<ipython-input-189-6e79e1b4ab52>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['modelName'] = new_df['modelName'].apply(clean_model_name)


In [193]:
new_df['modelName'] = new_df['modelName'].apply(lambda x: x.replace('logo', '').strip())

<ipython-input-193-0396f043b27a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['modelName'] = new_df['modelName'].apply(lambda x: x.replace('logo', '').strip())


In [197]:
new_df

,category,modelName,season,gender,colors
0,outerwear,hirmafur,cont,women,Black
1,outerwear,lans short down jacket,cont,women,Black
2,outerwear,grimpeurs hooded jacket,cont,men,Dark Blue
3,outerwear,maire,cont,women,Black
4,outerwear,lechtal down vest,ss24,men,Black
...,...,...,...,...,...
1734,soft accessories,ski helmet,fw23,unisex,Black
1735,soft accessories,ski helmet,cont,unisex,White
1736,soft accessories,padded gloves,cont,men,Black
1737,other goods,thermal bottle,cont,unisex,Black


In [198]:
##lets try to run the model without the logo term

data_encoded_n = pd.get_dummies(new_df, columns=['season', 'gender', 'colors','modelName'])


In [199]:
#SPLITTING DATA INTO train and test
from sklearn.model_selection import train_test_split


X = data_encoded_n.drop('category', axis=1)
y = data_encoded_n['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [200]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000))

# Fit the pipeline on the selected training data
pipeline.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=2000))])

In [201]:
from sklearn.metrics import accuracy_score

y_pred = pipeline.predict(X_test)

y_pred = pipeline.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.84


We can see the score improved with less noise in the data set going from .83 to .84